In [1]:
import os
import time
import pathlib

import numpy as np
import tensorflow as tf
import PIL.Image as Image
import tensorflow_hub as hub
import matplotlib.pylab as plt

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [16]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dir= pathlib.Path("/home/m3/Projects/Image-Clasification/Fruit_Vegetable_Classification/Dataset/V2/Train")
test_dir= pathlib.Path("/home/m3/Projects/Image-Clasification/Fruit_Vegetable_Classification/Dataset/V2/Test")

train_ds = image_dataset_from_directory(train_dir,shuffle=True,batch_size=BATCH_SIZE,image_size=IMG_SIZE)
test_ds = image_dataset_from_directory(test_dir,shuffle=True,batch_size=BATCH_SIZE,image_size=IMG_SIZE)

class_names = train_ds.class_names
n_class=len(class_names)

Found 3456 files belonging to 8 classes.
Found 768 files belonging to 8 classes.


In [17]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


In [37]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
dp=tf.keras.layers.Dropout(0.2)

In [29]:
base_model= hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",input_shape=(224,224,3),trainable=False)

In [40]:
model = tf.keras.Sequential([
  data_augmentation,
  rescale,
  base_model,
  dp,
  tf.keras.layers.Dense(n_class)
])

model.build([None, 224, 224, 3])

In [41]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_6 (Sequential)    (None, 224, 224, 3)       0         
_________________________________________________________________
rescaling_2 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
keras_layer_10 (KerasLayer)  (None, 1280)              4226432   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 10248     
Total params: 4,236,680
Trainable params: 10,248
Non-trainable params: 4,226,432
_________________________________________________________________


In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.
              metrics=['accuracy'])
